<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EfficientNet

# Table of contents

1. [Introduction to scaling Convnets](#1)
2. [Scaling dimensions](#2)
3. [Compound scaling method](#3)
4. [EfficientNet architecture](#4)
    1. [Inverted residual block](#4.1)
5. [EfficientNet-B0 implementation with Keras](#5)
    1. [Imports](#5.1)
    2. [Constants](#5.2)
    3. [Inverted residual block](#5.3)
    4. [EfficienNet-B0 network](#5.4)
6. [References](#6)

# Introduction to scaling ConvNets <a name="1"></a>

Convolutional Neural Networks (ConvNets or CNNs) have become ubiquitous in the world of Computer Vision. One of the key issues in designing CNNs, as with all other neural networks, is **model scaling**, i.e, deciding **how to increase the model size to achieve better accuracy**.

There are **many ways to scale a ConvNet** for different resource constraints: [ResNet](https://arxiv.org/abs/1512.03385) can be scaled down (ResNet-18) or up (ResNet-200) by adjusting **network depth (number of layers)**, while [WideResNet](https://arxiv.org/abs/1605.07146) and [MobileNets](https://arxiv.org/abs/1704.04861) can be scaled by **network width (number of channels)**. It is also known that a **bigger input image size (resolution)** will help accuracy with the overhead of more FLOPS (Floating Point Operations Per Second). Although prior studies have shown that network depth and width are both important for ConvNets power, it still remains an open question of how to effectively scale a ConvNet to achieve better efficiency and accuracy.

Google released a paper in 2019 that deal with a new family of CNNs, [**EfficientNet**](https://arxiv.org/abs/1905.11946). These CNNs not only provide better accuracy but also improve the efficiency of the models by reducing the parameters and FLOPS in comparison to the state of art models. Their study shows that it is critical to **balance all dimensions of network width/depth/resolution**, and such balance can be achieved by simply scaling each of them with a constant ratio. Based on this observation, they propose an effective ***compound scaling*** method. Unlike conventional practice that arbitrarily scales these factors, their method **uniformly scales network width, depth, and resolution** with a set of fixed scaling coefficients.  

# Scaling dimensions <a name="2"></a>

The main difficulty of this problem is that the optimal coefficients for **scaling network width, depth, and resolution depend on each other** and the values change under different resource constraints.  Due to this difficulty, **conventional methods** mostly **scale** ConvNets **in one of these dimensions**:

**Depth**: Scaling network depth is **the most common way** used by many ConvNets.  The intuition is that **deeper ConvNet can capture** richer and **more complex features**, and generalize well on new tasks. However, deeper networks are also **more difficult to train** due to the vanishing gradient problem. Although several techniques, such as skip connections and batch normalization, alleviate the training problem, the accuracy gain of very deep networks diminishes: for example, ResNet-1000 has similar accuracy as ResNet-101 even though it has much more layers. 

**Width**: Scaling network width is **commonly** used **for  small size models. Wider networks tend to capture more fine-grained features** and are **easier to train**. However, extremely wide but shallow networks tend to have **difficulties in capturing higher-level features** (the accuracy quickly saturates when networks become much wider than larger).

**Resolution**: With higher resolution input images, ConvNets can potentially **capture more fine-grained patterns**. Starting from $224 \times 224$ in early ConvNets,  modern ConvNets tend to use $299 \times  299$ or $331 \times 331$ for better accuracy.  Recently, [GPipe](https://arxiv.org/abs/1811.06965) achieves state-of-the-art ImageNet accuracy with $480 \times 480$ resolution. Higher resolutions, such as $600 \times 600$, are also widely used in object detection. Higher resolutions improve accuracy, but the accuracy gain diminishes for very high resolutions.

**Scaling up any dimension of network** width, depth, or resolution **improves accuracy, but the accuracy gain diminishes for bigger models**.

![](https://i.ibb.co/RH3WsYG/scale-up-dim-comvnets.png)

# Compound scaling method <a name="3"></a>

The authors observed that **different scaling dimensions are dependent**. Intuitively, for higher resolution images, we should increase network depth, such that the larger receptive fields can help capture similar features that include more pixels in bigger images. Correspondingly, we should also increase network width when the resolution is higher, in order to capture more fine-grained patterns with more pixels in high-resolution images. These intuitions suggest that we need to coordinate and **balance all scaling dimensions** rather than conventional single-dimension scaling.

They proposed a new **compound scaling method** to uniformly scales network width, depth, and resolution in a principled way. The method uses a compound coefficient which controls how many more resources are available for model scaling, and it's specified by the user.

Figure 2 illustrates the difference between the compound scaling method and conventional single-dimension scaling methods.

![](https://i.ibb.co/S6nvDbF/model-scaling.png)

**Problem formulation**

ConvNet layers are often partitioned into multiple stages or blocks, and all layers in each stage share the same architecture: for example, ResNet has five stages, and all layers in each stage have the same convolutional type (except the first layer which performs down-sampling).

Unlike regular ConvNet designs that mostly focus on finding the best layer architecture $F_i$, the authors tried to expand the network length ($L_i$), width ($C_i$), and/or resolution $(H_i,W_i)$ without changing $F_i$ predefined in the baseline network. By fixing $F_i$, model scaling simplifies the design problem for new resource constraints, but it still remains a large design space to explore different $L_i$, $C_i$, $H_i$, $W_i$ for each layer. In order to further reduce the design space, authors restricted that all layers must be scaled uniformly with a constant ratio.

# EfficientNet architecture <a name="4"></a>

The authors evaluated the scaling method using existing ConvNets (MovileNets, ResNets), but in order to better demonstrate the effectiveness of the scaling method, they also developed a new mobile-size baseline, called **EfficientNet**.

Inspired by [MnasNet](https://arxiv.org/abs/1807.11626), they develop a baseline network by leveraging a multi-objective neural architecture search that optimizes both accuracy and FLOPS. Table 1 shows the architecture of EfficientNet-B0. Its main building block is the inverted residual block MBConv ([MobileNetV2](https://arxiv.org/abs/1801.04381), [MnasNet](https://arxiv.org/abs/1807.11626)) which we discuss later. They also add [squeeze-and-excitation]((https://arxiv.org/abs/1709.01507)) optimization (SENet) (explained in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)).

![](https://i.ibb.co/wJNXbrH/architecture-efficientnet-b0.png)

Figure 8 compares the ImageNet performance of different scaling methods for the same EfficientNet-B0 baseline network. In general, all scaling methods improve accuracy with the cost of more FLOPS, but the compound scaling method can further improve accuracy, by up to 2.5%, than other single-dimension scaling methods, suggesting the importance of the proposed compound scaling.

![](https://i.ibb.co/MNFn87X/scaling-up-efficiennet.png)

### Inverted residual block <a name="4.1"></a>

An **inverted residual block**, sometimes called an MBConv block, is a **type of residual block** that uses an inverted structure for efficiency reasons. It was originally proposed for the [MobileNetV2](https://arxiv.org/abs/1801.04381) CNN architecture. It has since been reused for several mobile-optimized CNNs.

When we discussed the ResNet architecture (in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)) we saw that deep ResNets use a residual unit called **bottleneck residual block**. It has a `wide -> narrow -> wide` structure concerning the number of channels. That is, the input has a high number of channels, which are compressed with a $1 \times 1$ convolution. Then a $3 \times 3$ convolution. Finally, the number of channels is increased again with a $1 \times 1$ convolution so input and output can be added.

In contrast, an inverted residual block follows a `narrow -> wide -> narrow` approach, hence the inversion. We first widen with a $1 \times 1$ convolution, then we use a $3 \times 3$ **depthwise convolution** (which reduces the number of parameters) (depthwise convolution is explained in the notebook [CNN architectures](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb)), and finally we use a $1 \times 1$ convolution to reduce the number of channels so input and output can be added.

The next figure shows the difference between a bottleneck residual block and an inverted residual block:

![](https://i.ibb.co/x86q80k/inverted-residual-block.png)

Diagonally hatched layers do not use non-linearities. The thickness of each block is used to indicate its relative number of channels. Note how **classical residuals connect the layers with a high number of channels, whereas the inverted residuals connect the bottlenecks**.

# EfficientNet-B0 implementation with keras <a name="5"></a>

An implementation of EfficientNet B0 to B7 has been developed within tf.keras since TF2.3 ([source](https://keras.io/api/applications/efficientnet/)). Here, however, we are going to implement a simplified version of the EfficientNet-B0 network to understand it better. An example using this network for image classification and fine-tuning for a specific dataset is available in this  [kaggle's notebook](https://www.kaggle.com/viroviro/leaf-desease-classification). 

## Imports <a name="5.1"></a>

In [1]:
from keras.applications import imagenet_utils
from tensorflow.keras import layers, Model

## Constants <a name="5.2"></a>

We define a list of dictionaries with a collection of attributes to be used inside the inverted residual blocks.

In [2]:
DEFAULT_BLOCKS_ARGS = [{
    'kernel_size': 3,
    'layers_number': 1,
    'input_filters': 32,
    'output_filters': 16,
    'expand_ratio': 1,
    'strides': 1,
}, {
    'kernel_size': 3,
    'layers_number': 2,
    'input_filters': 16,
    'output_filters': 24,
    'expand_ratio': 6,
    'strides': 2,
}, {
    'kernel_size': 5,
    'layers_number': 2,
    'input_filters': 24,
    'output_filters': 40,
    'expand_ratio': 6,
    'strides': 2,
}, {
    'kernel_size': 3,
    'layers_number': 3,
    'input_filters': 40,
    'output_filters': 80,
    'expand_ratio': 6,
    'strides': 2,
}, {
    'kernel_size': 5,
    'layers_number': 3,
    'input_filters': 80,
    'output_filters': 112,
    'expand_ratio': 6,
    'strides': 1,
}, {
    'kernel_size': 5,
    'layers_number': 4,
    'input_filters': 112,
    'output_filters': 192,
    'expand_ratio': 6,
    'strides': 2,
}, {
    'kernel_size': 3,
    'layers_number': 1,
    'input_filters': 192,
    'output_filters': 320,
    'expand_ratio': 6,
    'strides': 1,
}]

## Inverted residual block <a name="5.3"></a>

In this section, we are going to implement the inverted residual block. But first, let's build a block which performs a 2D convolution, followed by batch normalization and swish activation function.

In [3]:
def conv_block(inputs, filters, kernel_size, strides, padding='same'):
    x = layers.Conv2D(
            filters=filters, kernel_size=kernel_size, strides=strides, 
            padding=padding, kernel_initializer=CONV_KERNEL_INITIALIZER
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('swish')(x)
    return x

As we discussed, an inverted residual block follows a `narrow -> wide -> narrow` approach. We first widen with a $1 \times 1$ convolution (expansion phase), then we use a $3 \times 3$ **depthwise convolution** (which reduces the number of parameters), and finally we use a $1 \times 1$ convolution to reduce the number of channels (reduction phase) so input and output can be added. After the depthwise convolution, Squeeze and Excitation is applied.

**Squeeze and Excitation Block**: In a CNN layer, the network weights each of its channels equally when creating the output feature maps. SENets add a mechanism to weight each channel adaptively. First, we get a global understanding of each channel by squeezing the feature maps to a single numeric value (through pooling). This results in a vector of size equal to the number of channels. Afterward, it is fed through a two-layer CNN, which outputs a vector of the same size. These values can now be used as weights on the original features maps, scaling each channel based on its importance. A further explanation about SENet is available [here](https://nbviewer.jupyter.org/github/victorviro/Deep_learning_python/blob/master/CNN_Architectures.ipynb#SENet-). 

In [4]:
def inverted_residual_block(
        inputs, activation='swish', output_filters=16, input_filters=32, 
        kernel_size=3, strides=1, expand_ratio=1, se_ratio=0.25, id_skip=True
    ):
    # Expansion phase
    filters_expanded = input_filters * expand_ratio
    x = conv_block(
        inputs=inputs, filters=filters_expanded, kernel_size=1, strides=1
    )

    # Depthwise Convolution
    x = layers.DepthwiseConv2D(
            kernel_size, strides=strides, padding='same', use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    # Squeeze and Excitation phase
    # Squeeze each feature map to a single numeric value 
    se = layers.GlobalAveragePooling2D()(x)
    se_shape = (1, 1, filters_expanded)
    se = layers.Reshape(se_shape)(se)
    # Two-layer CNN
    filters_se = max(1, int(input_filters * se_ratio))
    se = layers.Conv2D(
            filters_se, 1, padding='same', activation=activation
    )(se) 
    se = layers.Conv2D(
            filters_expanded, 1, padding='same', activation='sigmoid'
    )(se) 
    # Weight the original features maps
    x = layers.multiply([x, se])

    # Output phase (reduction phase)
    x = layers.Conv2D(
        output_filters, 1, padding='same', use_bias=False
    )(x) 
    x = layers.BatchNormalization()(x)

    # Add skip connection
    if id_skip and strides == 1 and input_filters == output_filters:
        x = layers.add([x, inputs])
    return x

This is a simplified implementation of an inverted residual block, the actual block is a bit more complex. It adds dropout regularization, specific kernel initialization, and it considers a few constraints ([source](https://github.com/keras-team/keras/blob/v2.6.0/keras/applications/efficientnet.py)).

## EfficientNet-B0 network <a name="5.4"></a>

![](https://i.ibb.co/wJNXbrH/architecture-efficientnet-b0.png)

In [5]:
def EfficientNetB0(input_shape=(None, None, 3), classes_number=1000, 
                   classifier_activation='softmax'):

    inputs = layers.Input(shape=input_shape)
    # Input preprocessing
    x = layers.Rescaling(1. / 255.)(inputs)
    x = layers.BatchNormalization()(x) 
    x = layers.ZeroPadding2D(padding=imagenet_utils.correct_pad(x, 3))(x)
    # First stage
    x = conv_block(x, filters=32, kernel_size=3, strides=2) 
    
    # Build blocks
    for (mbconv_block_index, mbconv_block_args) in enumerate(DEFAULT_BLOCKS_ARGS):

        for layer_number in range(mbconv_block_args['layers_number']):
            # Layers (except the first one) keep output shape unchanged 
            input_filters = mbconv_block_args['input_filters']
            strides = mbconv_block_args['strides']
            if layer_number > 0:
                strides = 1
                input_filters = mbconv_block_args['output_filters']
            x = inverted_residual_block(
                    inputs=x, 
                    output_filters=mbconv_block_args['output_filters'],
                    input_filters=input_filters,
                    kernel_size=mbconv_block_args['kernel_size'], 
                    strides=strides, 
                    expand_ratio=mbconv_block_args['expand_ratio']
            )
    # Build top
    x = conv_block(x, filters=1280, kernel_size=1, strides=1)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(
            classes_number,
            activation=classifier_activation
    )(x)
    # Create model
    model = Model(inputs, x, name='efficientnet-b0')
    return model

This is a simplified implementation of the EfficientNet-B0 network. The original implementation additionally adds dropout regularization, specific kernel initialization, etc ([source](https://github.com/keras-team/keras/blob/v2.6.0/keras/applications/efficientnet.py)).

# References <a name="6"></a>

- [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946)

- [EfficienNet module in TensorFlow](https://www.tensorflow.org/api_docs/python/tf/keras/applications/efficientnet)

- [EfficientNet B0 to B7 in Keras](https://keras.io/api/applications/efficientnet/)

- [Keras: Image classification via fine-tuning with EfficientNet](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/)

- [MobileNetV2: Inverted Residuals and Linear Bottlenecks](https://arxiv.org/abs/1801.04381)

- [paperswithcode: residual block](https://paperswithcode.com/method/residual-block)

- [paperswithcode: bottleneck residual block](https://paperswithcode.com/method/bottleneck-residual-block)

- [paperswithcode: inverted residual block](https://paperswithcode.com/method/inverted-residual-block)

